In [1]:
using Revise
using PauliPropagation

In [2]:
nq = 8

8

In [3]:
symbs = [:I for _ in 1:nq]
symbs[round(Int, nq/2)] = :Z   # as symbol. Also works but is slower.

obsint = symboltoint(symbs)  # for performance we work with bitoperations
show(obsint, nq)

IIIZIIII


In [4]:
nl = 4
topo = bricklayertopology(nq; periodic=false)
circ = hardwareefficientcircuit(nq, nl; topology=topo)
fastcirc = tofastgates(circ)
m = length(fastcirc)

124

In [5]:
using Random
Random.seed!(42)
thetas = randn(m);

In [6]:
W = Inf;                   # maximal operator weight.
min_abs_coeff = 0;          # neglect small coefficients. Only for numerical PP.

In [7]:
@time dnum = mergingbfs(fastcirc, obsint, thetas; max_weight=W, min_abs_coeff=min_abs_coeff);
@show length(dnum)   # number of unique Pauli ops

  0.590894 seconds (723.55 k allocations: 51.781 MiB, 1.81% gc time, 84.78% compilation time)
length(dnum) = 53247


53247

In [8]:
evalagainstzero(dnum) # expectation

0.21720058439757214